# Pydantic, Instructor and PydanticAI

- Pydantic <https://docs.pydantic.dev/latest/>
- Instrcutor <https://python.useinstructor.com>

## Start with Pydantic

In [2]:
from pydantic import BaseModel, ValidationError

# Define a Pydantic model
class User(BaseModel):
    name: str
    email: str
    age: int

try:
    valid_user = User(name="Alice", email="alice@example.com", age=25)
    print("valid user:", valid_user.dict())  # Convert model to dictionary
    invalid_user = User(name="Alice", email="alice@example.com", age="twenty five")

except ValidationError as e:
    print(e.json())  # Handle validation errors


valid user: {'name': 'Alice', 'email': 'alice@example.com', 'age': 25}
[{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"twenty five","url":"https://errors.pydantic.dev/2.10/v/int_parsing"}]


/var/folders/gb/kf8d65y549d33zybjv_4rww80000gn/T/ipykernel_53049/524425974.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print("valid user:", valid_user.dict())  # Convert model to dictionary


## Use Instructor to force structured Output

In [2]:
from openai import OpenAI
import instructor

client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)

# Define hook functions
def log_kwargs(**kwargs):
    print(f"Function called with kwargs: {kwargs}")


def log_exception(exception: Exception):
    print(f"An exception occurred: {str(exception)}")


client.on("completion:kwargs", log_kwargs)
client.on("completion:error", log_exception)


resp = client.chat.completions.create(
    model="llama3.2:latest",
    messages=[
        {
            "role": "user",
            "content": "Alice is 25 years old girl. She lives in Reutlingen and her email adress is alice@example.com",
        }
    ],
    response_model=User,
)
assert resp.name == "Alice"
assert resp.age == 25
assert resp.email == "alice@example.com"

Function called with kwargs: {'messages': [{'role': 'system', 'content': '\n        As a genius expert, your task is to understand the content and provide\n        the parsed objects in json that match the following json_schema:\n\n\n        {\n  "properties": {\n    "name": {\n      "title": "Name",\n      "type": "string"\n    },\n    "email": {\n      "title": "Email",\n      "type": "string"\n    },\n    "age": {\n      "title": "Age",\n      "type": "integer"\n    }\n  },\n  "required": [\n    "name",\n    "email",\n    "age"\n  ],\n  "title": "User",\n  "type": "object"\n}\n\n        Make sure to return an instance of the JSON, not the schema itself\n'}, {'role': 'user', 'content': 'Alice is 25 years old girl. She lives in Reutlingen and her email adress is alice@example.com'}], 'model': 'llama3.2:latest', 'response_format': {'type': 'json_object'}}


In [3]:
resp

User(name='Alice', email='alice@example.com', age=25)